In [ ]:
%pip install -Uqq fastai
%pip install -Uqq gradio
#|default_exp app

In [ ]:
#|export
from fastai.vision.all import *
import gradio as gr

def is_cat(x): return x[0].isupper()

In [ ]:
path = untar_data(URLs.PETS)/'images'

dls = ImageDataLoaders.from_name_func('.',
    get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_cat,
    item_tfms=Resize(192, method='squish'),
    # Passing in the Mac apple silicon device, instead of CPU
    device=default_device(1)
    )

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

In [ ]:
learn.export('model.pkl')

In [ ]:
#|export
learn = load_learner('model.pkl')

In [ ]:
#|export
# Below is fast.ai's code for gradio v3 but it's a few years old, and now there's gradio v5
# image = gr.inputs.Image(shape=(192, 192))
# label = gr.outputs.Label()
# examples = ['dog.jpg', 'cat.jpg', 'dunno.jpg']

# intf = gr.Interface(fn=classify_image, inputs=image, outputs=label, examples=examples)
# intf.launch(inline=False)

# Here's my gradio v5 fix
with gr.Blocks() as demo:
  gr.Markdown("Upload an image to see if it's a cat or a dog")
  image = gr.Image(height=192, width=192)
  button = gr.Button("Classify")
  label = gr.Label()

  button.click(fn=classify_image, inputs=image, outputs=label)

demo.launch(inline=False)


In [ ]:
# Preparing the Gradio code (cells prefixed with |export) for being deployed to Hugging Face

# This part is from fast.ai's code but it's a few years old, updated it below
# from nbdev.export import notebook2script
# notebook2script('app.ipynb')

# My fix
import nbdev
nbdev.export.nb_export('app.ipynb', 'generated', name='huggingface_app')
print('Export successful')
